In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.adam import Adam
from torch.nn.modules import Linear
from torch.nn.modules import Sequential

In [ ]:
from random import uniform as rf
import math
import numpy as np

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def get_sample(n):
    x=np.random.uniform(-1,1,n*2).reshape(n,2)
    y=np.where(np.sqrt(np.sum(np.square(x),axis=1))<0.75,1,0).reshape(n,1)
    #y=.5*np.sum(x,axis=1)
    return torch.from_numpy(x.astype('float32')).cuda(),torch.from_numpy(y.astype('float32')).cuda()

In [ ]:
class Model(nn.Module):
    def __init__(self, din, hid, dout):
        super(Model, self).__init__()
        self.linear1=Linear(din,hid)
        self.linear2=Linear(hid,dout)
        self.tanh=nn.Tanh()
        self.sigmoid=nn.Sigmoid()
    def forward(self, x):
        y=self.tanh(self.linear1(x))
        y=self.sigmoid(self.linear2(y))
        return y

In [ ]:
model=Model(2,4,1).cuda()
optim=Adam(model.parameters(),.01)

In [ ]:
def train(n):
    x,y=get_sample(n)
    y_pred=model(x)
    loss=F.mse_loss(y_pred,y)
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    return loss,y,y_pred

In [ ]:
for i in range(10000):
    loss,y_real,y_pred=train(1000)
    if i % 1000 == 0:
        print(loss)

In [ ]:
x,y=get_sample(1000)
y_pred=model(x)

fig = plt.figure()
ax = Axes3D(fig)

ax.scatter(x.cpu().numpy()[:,0:1],x.cpu().numpy()[:,1:2],y.cpu().detach().numpy())
plt.show()

fig2 = plt.figure()
ax = Axes3D(fig2)

ax.scatter(x.cpu().numpy()[:,0:1],x.cpu().numpy()[:,1:2],y_pred.cpu().detach().numpy())
plt.show()